# Using cursors in the Agrimetrics Graph API

When making queries which could return large numbers of objects (e.g. "all fields within 100km of this point", or "this location's daily weather readings for the last 10 years"), it can be useful, or even sometimes necessary, to split the query across multiple requests, with each request retrieving the next part of the result.

In the Agrimetrics Graph Query API, this function is performed by *cursors*. A cursor contains the state of a query, such that the same query can be resumed from that point. So, to make a large query, start by making the query, and ask for the `cursor` property to be returned alongside the data you want to retrieve. Then make the same query again, but pass the (last) `cursor` property you retrieved in the results back in as the `after` parameter, and you will get the results immediately following the last result of the previous query. This process can be repeated as many times as necessary, until the `cursor` property becomes `null`.

We start this example by importing the usual libraries.

In [1]:
import json
import os
from pprint import pprint
import requests

import pandas as pd

Define the GraphQL endpoint and the API subscription key.

In [2]:
GRAPHQL_ENDPOINT = "https://api.agrimetrics.co.uk/graphql/v1/"

if "API_KEY" in os.environ:
    API_KEY = os.environ["API_KEY"]
else:
    API_KEY = input("Query API Subscription Key: ").strip()
    
common_headers = {
    "OCP-APIM-Subscription-Key": API_KEY
}

For this example, we will be searching for fields in an area around a point, and retrieving them in groups of 20.

In [3]:
POINT_LOCATION = {
    "type": "Point",
    "coordinates": [-0.368728, 51.809994]
}
QUERY_DISTANCE = 3500
BLOCK_SIZE = 20

We define our query with a `$first` parameter to define the number of items to return in each result set, and an optional `$after` parameter for passing the cursor in. The `$after` parameter will be `null` (`None` in Python) on the first call.

In [4]:
fields_query = """
  query FieldsQuery($location: LocationFilter!, $range: Float!, $after: String, $first: Int!) {
    fields(
      geoFilter: { location: $location, distance: {LE: $range} }
      after: $after
      first: $first
    ) {
      id
      cursor
      soil {
        topSoil {
          texture {
            type
          }
        }
      }
    }
  }
"""

We can now loop, making the query until the `cursor` field is `null` (or, in the case of Python, `None`). We accumulate the results in a data frame.

Note the last part of the loop, where the loop exits if the new `cursor` value is `None`, and then the `after` variable is updated with the new `cursor` value.

In [5]:
fields_variables = {
  "location": POINT_LOCATION,
  "range": QUERY_DISTANCE,
  "first": BLOCK_SIZE
}
fields_query_object = {
  "query": fields_query,
  "variables": fields_variables,
  "operationName": "FieldsQuery"
}

data_frame = pd.DataFrame()

while True:
    # Get the next batch of data
    result = requests.post(GRAPHQL_ENDPOINT, headers=common_headers, json=fields_query_object)
    result.raise_for_status()
    data = result.json()["data"]

    # Extract the new data and add it to the data frame
    field_data = ((field["id"], field["soil"]["topSoil"]["texture"]["type"]) for field in data["fields"])
    current_frame = pd.DataFrame(field_data)
    data_frame = data_frame.append(current_frame)

    # Check if we've reached the end of the cursor: the last field has a null cursor
    cursor = data["fields"][-1]["cursor"]
    if cursor is None:
        break
    # We have a non-null cursor. Continue from that point.
    fields_variables["after"] = cursor

data_frame

,0,1
0,https://data.agrimetrics.co.uk/fields/-51RfbWi...,LOAM
1,https://data.agrimetrics.co.uk/fields/-7bU1VSs...,LOAM
2,https://data.agrimetrics.co.uk/fields/-G1GIVD7...,LOAM
3,https://data.agrimetrics.co.uk/fields/-HpNw3N6...,CLAY_LOAM
4,https://data.agrimetrics.co.uk/fields/-_Ds2q4S...,CLAY_LOAM
...,...,...
17,https://data.agrimetrics.co.uk/fields/yqqWuxdQ...,CLAY_LOAM
18,https://data.agrimetrics.co.uk/fields/z8CF6Hrd...,LOAM
19,https://data.agrimetrics.co.uk/fields/zCA-KmXJ...,CLAY_LOAM
0,https://data.agrimetrics.co.uk/fields/zVOVJWpt...,CLAY_LOAM


## Other cursors

Similar kinds of cursoring may also appear elsewhere in the API. For example, weather observations can also be cursored over using the same approach. With weather observations, the cursor value is *outside* the individual readings, and represents a sub-range of the original date period.

In [6]:
FIELD_ID = "https://data.agrimetrics.co.uk/fields/C6BgTxUxhMG_OCGrLGW8qw"
START_DATE = "2018-01-01"
END_DATE = "2018-12-31"

weather_query = """
  query WeatherQuery(
    $id: [ID!]!,
    $startDate: Date!,
    $endDate: Date!,
    $after: String
  ) {
    fields(where: { id: {EQ: $id}}) {
      id
      weatherObservations(
        where: {
          date: {GE: $startDate, LE: $endDate}
        },
        after: $after
      ) {
        cursor
        rainfallTotalDaily {
          dateTime
          value
        }
        hoursOfSunshineMonthly {
          dateTime
          value
        }
      }
    }
  }
"""

Now, set up the intial variables and query structures, and a data frame to accumulate the results into.

In [7]:
weather_variables = {
  "id": FIELD_ID,
  "startDate": START_DATE,
  "endDate": END_DATE
}
weather_query_object = {
  "query": weather_query,
  "variables": weather_variables,
  "operationName": "WeatherQuery"
}

data_frame = pd.DataFrame()

And finally, iterate to retrieve readings until no more are available, and the `cursor` property becomes `null`.

In [8]:
while True:
    # Get the next batch of data
    result = requests.post(GRAPHQL_ENDPOINT, headers=common_headers, json=weather_query_object)
    result.raise_for_status()
    data = result.json()["data"]
    weather = data["fields"][0]["weatherObservations"]

    # Extract the new data and add it to the data frame
    rain_data = pd.DataFrame(weather["rainfallTotalDaily"])
    sun_data = pd.DataFrame(weather["hoursOfSunshineMonthly"])
    current_frame = pd.merge(rain_data, sun_data, on="dateTime", how="outer")
    current_frame = current_frame.rename(columns={"value_x": "rainfallTotalDaily", "value_y": "hoursOfSunshineMonthly"})
    data_frame = data_frame.append(current_frame)

    # Check if we've reached the end of the cursor: the last field has a null cursor
    cursor = weather["cursor"]
    if cursor is None:
        break
    # We have a non-null cursor. Continue from that point.
    weather_variables["after"] = cursor

Print out the combined results. Note that we're joining on date, and we have one daily series and one monthly series, so most of the monthly values are empty (`NaN`).

In [9]:
print(data_frame.to_string())

                     dateTime  rainfallTotalDaily  hoursOfSunshineMonthly
0    2018-01-01T00:00:00.000Z                 0.9                   60.22
1    2018-01-02T00:00:00.000Z                 6.7                     NaN
2    2018-01-03T00:00:00.000Z                 3.5                     NaN
3    2018-01-04T00:00:00.000Z                 0.8                     NaN
4    2018-01-05T00:00:00.000Z                 0.0                     NaN
5    2018-01-06T00:00:00.000Z                 0.2                     NaN
6    2018-01-07T00:00:00.000Z                 0.0                     NaN
7    2018-01-08T00:00:00.000Z                 0.0                     NaN
8    2018-01-09T00:00:00.000Z                 0.4                     NaN
9    2018-01-10T00:00:00.000Z                 4.5                     NaN
10   2018-01-11T00:00:00.000Z                 0.6                     NaN
11   2018-01-12T00:00:00.000Z                 0.0                     NaN
12   2018-01-13T00:00:00.000Z         